In [248]:
import os
import gc
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns

import time
from sklearn.model_selection import train_test_split
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)

In [249]:
train = pd.read_csv("train_fwYjLYX.csv", parse_dates=['application_date'])
test = pd.read_csv("test_1eLl9Yf.csv", parse_dates=['application_date'])
submission = pd.read_csv("sample_submission_IIzFVsf.csv", parse_dates=['application_date'])
train.shape,test.shape,submission.shape

((80402, 6), (180, 3), (180, 4))

In [250]:
test.drop(['id'], axis=1, inplace=True)
train = train.sort_values('application_date').reset_index(drop = True)
test = test.sort_values('application_date').reset_index(drop = True)

In [251]:
train.application_date.min(), train.application_date.max()

(Timestamp('2017-04-01 00:00:00'), Timestamp('2019-07-23 00:00:00'))

In [252]:
test.application_date.min(), test.application_date.max()

(Timestamp('2019-07-06 00:00:00'), Timestamp('2019-10-24 00:00:00'))

In [253]:
agg_func = {'case_count': ['sum']}
agg_name = train.groupby(['segment','application_date']).agg(agg_func)
agg_name.columns = [ 'SA_' + ('_'.join(col).strip()) for col in agg_name.columns.values]
agg_name.reset_index(inplace=True)
train = train.merge(agg_name, on=['segment','application_date'], how='left')
del agg_name
train.drop(['branch_id','state','zone','case_count'], axis=1, inplace=True)
train = train.rename(columns={'SA_case_count_sum': 'case_count'})
train.drop_duplicates(keep='first', inplace=True)
train = train.sort_values('application_date').reset_index(drop = True)
# df = train.append(test, ignore_index=True,sort=False)

train['train_or_test'] = 'train'
test['train_or_test'] = 'test'
df = pd.concat([train,test], sort=False)
print('Combined df shape:{}'.format(df.shape))
del train, test
gc.collect()

Combined df shape:(1830, 4)


264

In [254]:
df.head(2)

,application_date,segment,case_count,train_or_test
0,2017-04-01,1,299.0,train
1,2017-04-01,2,897.0,train


In [255]:
# Extracting application_date features
df['dayofmonth'] = df.application_date.dt.day
df['dayofyear'] = df.application_date.dt.dayofyear
df['dayofweek'] = df.application_date.dt.dayofweek
df['month'] = df.application_date.dt.month
df['year'] = df.application_date.dt.year
df['weekofyear'] = df.application_date.dt.weekofyear
df['is_month_start'] = (df.application_date.dt.is_month_start).astype(int)
df['is_month_end'] = (df.application_date.dt.is_month_end).astype(int)
df.head()

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,299.0,train,1,91,5,4,2017,13,1,0
1,2017-04-01,2,897.0,train,1,91,5,4,2017,13,1,0
2,2017-04-02,2,605.0,train,2,92,6,4,2017,13,0,0
3,2017-04-03,1,42.0,train,3,93,0,4,2017,14,0,0
4,2017-04-03,2,2016.0,train,3,93,0,4,2017,14,0,0


In [256]:
df.sort_values(by=['segment','application_date'], axis=0, inplace=True)
df.head(2)

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,299.0,train,1,91,5,4,2017,13,1,0
3,2017-04-03,1,42.0,train,3,93,0,4,2017,14,0,0


In [257]:
# Features constructed from previous case_count values

# Creating case_count lag features
def create_case_count_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = \
                gpby[target_col].shift(i).values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating case_count rolling mean features
def create_case_count_rmean_feats(df, gpby_cols, target_col, windows, min_periods=2, 
                             shift=1, win_type=None):
    gpby = df.groupby(gpby_cols)
    for w in windows:
        df['_'.join([target_col, 'rmean', str(w)])] = \
            gpby[target_col].shift(shift).rolling(window=w, 
                                                  min_periods=min_periods,
                                                  win_type=win_type).mean().values + np.random.normal(scale=1.6, size=(len(df),))
    return df

# Creating case_count exponentially weighted mean features
def create_case_count_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = \
                gpby[target_col].shift(s).ewm(alpha=a).mean().values + np.random.normal(scale=1.6, size=(len(df),))
    return df

In [258]:
# OHE of categorical features

def one_hot_encoder(df, ohe_cols=['dayofmonth','dayofweek','month','weekofyear']):
    '''
    One-Hot Encoder function
    '''
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df = pd.get_dummies(df, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df

In [259]:
# Converting case_count to log(1+case_count)
df['case_count'] = np.log1p(df.case_count.values)
df.head(2)

,application_date,segment,case_count,train_or_test,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,2017-04-01,1,5.703782,train,1,91,5,4,2017,13,1,0
3,2017-04-03,1,3.761200,train,3,93,0,4,2017,14,0,0


In [260]:
# Time-based Validation set

# For validation to keep months also identical to test set we can choose period (same of 2018) as the validation set.

masked_series = (df['application_date'] >= '2018-07-06') & (df['application_date'] <= '2018-10-24')
masked_series2 = (df['application_date'] < '2018-07-06') & (df['application_date'] > '2018-10-24')
df.loc[(masked_series), 'train_or_test'] = 'val'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df.train_or_test=='train',:].shape))
print('Validation shape: {}'.format(df.loc[df.train_or_test=='val',:].shape))
print('No train shape: {}'.format(df.loc[df.train_or_test=='no_train',:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))

Train shape: (1428, 12)
Validation shape: (222, 12)
No train shape: (0, 12)
Test shape: (180, 12)


In [261]:
# Model Validation

# Converting case_count of validation period to nan so as to resemble test period
train = df.loc[df.train_or_test.isin(['train','val']), :]
Y_val = train.loc[train.train_or_test=='val', 'case_count'].values.reshape((-1))
Y_train = train.loc[train.train_or_test=='train', 'case_count'].values.reshape((-1))
train.loc[train.train_or_test=='val', 'case_count'] = np.nan

# # Creating case_count lag, rolling mean, rolling median, ohe features of the above train set
train = create_case_count_lag_feats(train, gpby_cols=['segment'], target_col='case_count', 
#                                lags=[91,98,105,112,119,126,133,140,147,154,161,168,175,182,364,546,728])
                                    lags=[91,98,105,112,119,126,182,364,546,728])

train = create_case_count_rmean_feats(train, gpby_cols=['segment'], 
                                 target_col='case_count', windows=[364,546], 
                                 min_periods=10, win_type='triang') #98,119,91,182,


train = create_case_count_ewm_feats(train, gpby_cols=['segment'], 
                               target_col='case_count', 
                               alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5], 
                               shift=[91,98,105,112,119,126,182,364,546,728])


# One-Hot Encoding 
train = one_hot_encoder(train, ohe_cols=['dayofweek','month'])

# Final train and val datasets
val = train.loc[train.train_or_test=='val', :]
train = train.loc[train.train_or_test=='train', :]
print('Train shape:{}, Val shape:{}'.format(train.shape, val.shape))

Creating OHE features..
Old df shape:(1650, 84)
New df shape:(1650, 101)
Train shape:(1428, 101), Val shape:(222, 101)


In [262]:
avoid_cols = ['application_date', 'case_count', 'train_or_test', 'id', 'year','is_month_start']
cols = [col for col in train.columns if col not in avoid_cols]
print('No of training features: {} \nAnd they are:{}'.format(len(cols), cols))

No of training features: 96 
And they are:['segment', 'dayofmonth', 'dayofyear', 'weekofyear', 'is_month_end', 'case_count_lag_91', 'case_count_lag_98', 'case_count_lag_105', 'case_count_lag_112', 'case_count_lag_119', 'case_count_lag_126', 'case_count_lag_182', 'case_count_lag_364', 'case_count_lag_546', 'case_count_lag_728', 'case_count_rmean_364', 'case_count_rmean_546', 'case_count_lag_91_ewm_0.95', 'case_count_lag_98_ewm_0.95', 'case_count_lag_105_ewm_0.95', 'case_count_lag_112_ewm_0.95', 'case_count_lag_119_ewm_0.95', 'case_count_lag_126_ewm_0.95', 'case_count_lag_182_ewm_0.95', 'case_count_lag_364_ewm_0.95', 'case_count_lag_546_ewm_0.95', 'case_count_lag_728_ewm_0.95', 'case_count_lag_91_ewm_0.9', 'case_count_lag_98_ewm_0.9', 'case_count_lag_105_ewm_0.9', 'case_count_lag_112_ewm_0.9', 'case_count_lag_119_ewm_0.9', 'case_count_lag_126_ewm_0.9', 'case_count_lag_182_ewm_0.9', 'case_count_lag_364_ewm_0.9', 'case_count_lag_546_ewm_0.9', 'case_count_lag_728_ewm_0.9', 'case_count_lag_9

In [263]:
def mape(preds, target):
    '''
    Function to calculate MAPE
    '''
    n = len(preds)
    masked_arr = ~((preds==0)&(target==0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = target-preds
    denom = target
    mape_val = np.mean(np.abs(num / denom)) * 100
    return mape_val

def lgbm_mape(preds, train_data):
    '''
    Custom Evaluation Function for LGBM
    '''
    labels = train_data.get_label()
    mape_val = mape(np.expm1(preds), np.expm1(labels))
    return 'MAPE', mape_val, False

In [264]:
# LightGBM parameters
lgb_params = {'task':'train', 'boosting_type':'gbdt', 'objective':'mape', 
              'learning_rate': 0.02, 'verbose': 0, 'num_leaves': 62,
              'num_boost_round':30000, 'early_stopping_rounds':2000, 'nthread':-1, 'random_state':0}

In [265]:
# Creating lgbtrain & lgbval
lgbtrain = lgb.Dataset(data=train.loc[:,cols].values, label=Y_train, 
                       feature_name=cols)
lgbval = lgb.Dataset(data=val.loc[:,cols].values, label=Y_val, 
                     reference=lgbtrain, feature_name=cols)

In [266]:
def lgb_validation(params, lgbtrain, lgbval, X_val, Y_val, verbose_eval):
    t0 = time.time()
    evals_result = {}
    model = lgb.train(params, lgbtrain, num_boost_round=params['num_boost_round'], 
                      valid_sets=[lgbtrain, lgbval], feval=lgbm_mape, 
                      early_stopping_rounds=params['early_stopping_rounds'], 
                      evals_result=evals_result, verbose_eval=verbose_eval)
    print(model.best_iteration)
    print('Total time taken to build the model: ', (time.time()-t0)/60, 'minutes!!')
    pred_Y_val = model.predict(X_val, num_iteration=model.best_iteration)
    pred_Y_val = np.expm1(pred_Y_val)
    Y_val = np.expm1(Y_val)
    val_df = pd.DataFrame(columns=['true_Y_val','pred_Y_val'])
    val_df['pred_Y_val'] = pred_Y_val
    val_df['true_Y_val'] = Y_val
    print(val_df.shape)
    print(val_df.head(5))
    print('MAPE for validation data is:{}'.format(mape(pred_Y_val, Y_val)))
    return model, val_df

In [267]:
# Training lightgbm model and validating
model, val_df = lgb_validation(lgb_params, lgbtrain, lgbval, val.loc[:,cols].values, 
                               Y_val, verbose_eval=500)
# 49.88 1429

Training until validation scores don't improve for 2000 rounds.
[500]	training's mape: 0.0261623	training's MAPE: 22.8973	valid_1's mape: 0.0460621	valid_1's MAPE: 50.6978
[1000]	training's mape: 0.0229222	training's MAPE: 19.1855	valid_1's mape: 0.046194	valid_1's MAPE: 50.648
[1500]	training's mape: 0.0223243	training's MAPE: 17.6642	valid_1's mape: 0.0460103	valid_1's MAPE: 49.9977
[2000]	training's mape: 0.0230312	training's MAPE: 17.3115	valid_1's mape: 0.0473784	valid_1's MAPE: 51.8376
[2500]	training's mape: 0.0226699	training's MAPE: 15.8324	valid_1's mape: 0.0476591	valid_1's MAPE: 51.2734
[3000]	training's mape: 0.022277	training's MAPE: 15.3651	valid_1's mape: 0.0476116	valid_1's MAPE: 51.4516
Early stopping, best iteration is:
[1429]	training's mape: 0.0215806	training's MAPE: 17.489	valid_1's mape: 0.0459044	valid_1's MAPE: 49.8886
1429
Total time taken to build the model:  1.5161173423131307 minutes!!
(222, 2)
   true_Y_val   pred_Y_val
0      2890.0  1782.688185
1      2

In [268]:
# Let's see top 25 features as identified by the lightgbm model.
print("Features importance...")
gain = model.feature_importance('gain')
feat_imp = pd.DataFrame({'feature':model.feature_name(), 
                         'split':model.feature_importance('split'), 
                         'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print('Top 25 features:\n', feat_imp.head(50))

Features importance...
Top 25 features:
                         feature  split      gain
1                    dayofmonth   3600  6.533533
60   case_count_lag_112_ewm_0.6   1541  2.644944
13           case_count_lag_546    827  2.366966
17   case_count_lag_91_ewm_0.95   1364  2.358652
8            case_count_lag_112   1194  2.230805
30   case_count_lag_112_ewm_0.9   1195  2.103275
15         case_count_rmean_364   2305  2.045164
70   case_count_lag_112_ewm_0.5   1219  2.033752
0                       segment    195  2.028177
40   case_count_lag_112_ewm_0.8   1148  2.026639
20  case_count_lag_112_ewm_0.95   1176  1.942020
2                     dayofyear   2161  1.907056
16         case_count_rmean_546   1868  1.661541
41   case_count_lag_119_ewm_0.8   1122  1.555234
65   case_count_lag_546_ewm_0.6    714  1.471248
45   case_count_lag_546_ewm_0.8    844  1.461482
29   case_count_lag_105_ewm_0.9   1011  1.461074
5             case_count_lag_91   1346  1.437954
6             case_count_lag

In [269]:
# Let's see top 25 features as identified by the lightgbm model.
print("Features importance...")
gain = model.feature_importance('gain')
feat_imp = pd.DataFrame({'feature':model.feature_name(), 
                         'split':model.feature_importance('split'), 
                         'gain':100 * gain / gain.sum()}).sort_values('split', ascending=False)
print('Top 25 features:\n', feat_imp.head(500))

Features importance...
Top 25 features:
                         feature  split      gain
1                    dayofmonth   3600  6.533533
15         case_count_rmean_364   2305  2.045164
2                     dayofyear   2161  1.907056
16         case_count_rmean_546   1868  1.661541
60   case_count_lag_112_ewm_0.6   1541  2.644944
17   case_count_lag_91_ewm_0.95   1364  2.358652
5             case_count_lag_91   1346  1.437954
6             case_count_lag_98   1255  1.415010
47    case_count_lag_91_ewm_0.7   1240  1.336415
70   case_count_lag_112_ewm_0.5   1219  2.033752
30   case_count_lag_112_ewm_0.9   1195  2.103275
8            case_count_lag_112   1194  2.230805
20  case_count_lag_112_ewm_0.95   1176  1.942020
33   case_count_lag_182_ewm_0.9   1149  1.414887
40   case_count_lag_112_ewm_0.8   1148  2.026639
11           case_count_lag_182   1143  1.153706
63   case_count_lag_182_ewm_0.6   1138  1.357587
41   case_count_lag_119_ewm_0.8   1122  1.555234
27    case_count_lag_91_ewm_

In [270]:
# Creating case_count lag, rolling mean, rolling median, ohe features of the above train set
df_whole = create_case_count_lag_feats(df, gpby_cols=['segment'], target_col='case_count', 
                                  lags=[91,98,105,112,119,126,182,364,546,728])

df_whole = create_case_count_rmean_feats(df_whole, gpby_cols=['segment'], 
                                    target_col='case_count', windows=[364,546], 
                                    min_periods=10, win_type='triang')

df_whole = create_case_count_ewm_feats(df_whole, gpby_cols=['segment'], target_col='case_count', 
                                  alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5], 
                                  shift=[91,98,105,112,119,126,182,364,546,728])



# One-Hot Encoding
df_whole = one_hot_encoder(df_whole, ohe_cols=['dayofweek','month']) 

# Final train and test datasets
test = df_whole.loc[df_whole.train_or_test=='test', :]
train = df_whole.loc[~(df_whole.train_or_test=='test'), :]
print('Train shape:{}, Test shape:{}'.format(train.shape, test.shape))

Creating OHE features..
Old df shape:(1830, 84)
New df shape:(1830, 101)
Train shape:(1650, 101), Test shape:(180, 101)


In [271]:
from fbprophet import Prophet
from tqdm import tqdm, tqdm_notebook
import datetime as dt

def create_prophet_features(train):
    """
    Train a seperate fbprophet model on each segment.
    Use the results as features used by the final model
    """
    grouped = train.groupby(['segment'])
    prophet_results = []
    for i, d in tqdm(grouped):
        m = Prophet(uncertainty_samples=100, daily_seasonality=False)
        m.fit(d.rename(columns={'application_date': 'ds', 'case_count': 'y'}))
        future = m.make_future_dataframe(periods=90)
        forecast = m.predict(future)
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
        forecast['segment'] = i
        prophet_results.append(forecast)
    prophet_features = pd.concat(prophet_results)
    prophet_features = prophet_features.rename(columns={'ds': 'application_date'})
    prophet_features.to_csv('prophet_results.csv')
    return prophet_features


pro = create_prophet_features(train)
corr_col = ['multiplicative_terms','multiplicative_terms_lower','multiplicative_terms_upper',
            'additive_terms_lower','additive_terms_upper','weekly_lower','weekly_upper',
           'yearly_lower','yearly_upper','trend_lower','trend_upper']
final_cols = [col for col in pro.columns if col not in corr_col]
print(pro[final_cols].shape)
pro[final_cols].reset_index(drop=True,inplace=True)

train = train.merge(pro[final_cols], on=['segment','application_date'], how='left')
test = test.merge(pro[final_cols], on=['segment','application_date'], how='left')
print(train.shape,test.shape)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.61s/it]


(1830, 9)
(1650, 108) (180, 108)


In [272]:
# LightGBM parameters
lgb_params = {'task':'train', 'boosting_type':'gbdt', 'objective':'mape', 
              'metric': 'mape', 'learning_rate': 0.02, 'verbose': 0, 'num_leaves': 62,
              'num_boost_round':model.best_iteration*1.2,'nthread':-1, 'random_state':0}

In [273]:
# LightGBM dataset
lgbtrain_all = lgb.Dataset(data=train.loc[:,cols].values, 
                           label=train.loc[:,'case_count'].values.reshape((-1,)), 
                           feature_name=cols)

In [274]:
def lgb_train(params, lgbtrain_all, X_test, num_round):
    t0 = time.time()
    model = lgb.train(params, lgbtrain_all, num_boost_round=num_round, feval=lgbm_mape)
    test_preds = model.predict(X_test, num_iteration=num_round)
    print('Total time taken in model training: ', (time.time()-t0)/60, 'minutes!')
    return model, test_preds

In [275]:
# Training lgb model on whole data(train+val)
lgb_model, test_preds = lgb_train(lgb_params, lgbtrain_all, test.loc[:,cols].values, model.best_iteration)
print('test_preds shape:{}'.format(test_preds.shape))

Total time taken in model training:  0.8421529690424602 minutes!
test_preds shape:(180,)


In [276]:
submission['case_count'] = np.expm1(test_preds)
submission.to_csv('lgbm_V30.csv', index=False)